# **Setup**

1. Dependencies Installieren

In [ ]:
!pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

2. Package importieren

In [9]:
#Packung für das Sammeln durch den Webcam
import os, time, uuid, cv2
from pathlib import Path

In [2]:
#Packung für Bilderverarbeitung
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

In [3]:
#Packung für die Ablumentation und die Bildung der Data-Pipeline
import albumentations as alb

In [4]:
#Packung für das Modelbau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [5]:
#Model Speicherung
from tensorflow.keras.models import load_model

# Bilder mit optimaler Namen umnennen.

In [11]:
# Da ich mit Google Colab arbeiten, ist die Name meines Paths anderes bei euch. Allerdings ist dieser Teil nur für mich wichtig, deshalb musst ihr dieses Teil auch nicht activieren.
%cd "/content/drive/MyDrive/Programming/KIprojekt/BilderkennungAI"

/content/drive/MyDrive/Programming/KIprojekt/BilderkennungAI


In [7]:
image_path = "data/images"

In [14]:
%cd "/content/drive/MyDrive/Programming/KIprojekt/BilderkennungAI/{image_path}"

/content/drive/MyDrive/Programming/KIprojekt/BilderkennungAI/data/images


In [15]:
current_path = Path().resolve()
print(current_path)

/content/drive/MyDrive/Programming/KIprojekt/BilderkennungAI/data/images


In [17]:
x = 1
for file in os.listdir():
  old_name = os.path.join(current_path, file)
  new_name = os.path.join(current_path,f"image{x}.jpg")
  os.rename(old_name, new_name)
  x+=1